In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
# Loading dataframe
data = pd.read_csv("IMDB Dataset.csv")
data.sample(10)

,review,sentiment
6164,The finale of the Weissmuller Tarzan movies is...,negative
40667,When an actor has to play the role of an actor...,positive
43871,I saw an early screening of this film in New Y...,negative
44680,I read the novel some years ago and I liked it...,negative
16236,I admit not being that fond of Oliver! as a yo...,positive
7687,Watched this as a late TV movie last night pur...,positive
49899,After realizing what is going on around us ......,positive
33220,"The storyline of ""The Stranger"" mirrors somewh...",negative
37719,This is by far the funniest short made by the ...,positive
9417,Oz is the TV show which is intensive non-stop ...,positive


In [3]:
# Make 0 represent negative and 1 represent positive reivews
data['label'] = data['sentiment'].replace(['positive', 'negative'],['1', '0'])

In [4]:
data.sample(10)

,review,sentiment,label
39503,This movie was recommended to me by several pe...,negative,0
37576,We all enjoyed the movie. It is a very charmin...,positive,1
10307,I keep waiting for Peter Fonda to start acting...,negative,0
8911,This movie is even a big step down form the ty...,negative,0
17917,Minnie and Moskowitz is the most pathetic and ...,positive,1
14874,I fell in love with Emily Watson in Breaking t...,positive,1
36168,Many have disparaged Never Say Never Again bec...,positive,1
6781,"Absolutely unwatchable, lowest quality film ma...",negative,0
11161,"When I went to the cinema, I expected not much...",positive,1
43885,Hidden Frontier is notable for being the longe...,positive,1


In [5]:
data['label'].value_counts()

label
1    25000
0    25000
Name: count, dtype: int64

In [6]:
data.to_csv(r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\\Assignment\Assignment 1\IMDB_Dataset_label.csv')

In [7]:
#Text Preprocessing for sentiment analysis
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
    
    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    #2. clean the number 
    text = re.sub(r'[0-9]', '', text)
    
    #3. lower the text
    text = text.lower()
    
    #4. conver the emoji to text form
    text = emoji.demojize(text)
    
    #5. remove punctuation 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    #6. tokenize the text
    text = word_tokenize(text)
    
    #7. remove empty token
    text = [t for t in text if len(t) > 0]
    
    #8. remove non-alphabetical token
    text = [t for t in text if t.isalpha()]
    
    #9. replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    #10. remove the stopwords
    text = [i for i in text if i not in stopwords]
    
    #11. stem the text
    porter_stemmer = PorterStemmer()
    text = [porter_stemmer.stem(w) for w in text]
    
    return text

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [8]:
print(data['review'].iloc[12])
print(preprocess(data['review'].iloc[12]))

So im not a big fan of Boll's work but then again not many are. I enjoyed his movie Postal (maybe im the only one). Boll apparently bought the rights to use Far Cry long ago even before the game itself was even finsished. <br /><br />People who have enjoyed killing mercs and infiltrating secret research labs located on a tropical island should be warned, that this is not Far Cry... This is something Mr Boll have schemed together along with his legion of schmucks.. Feeling loneley on the set Mr Boll invites three of his countrymen to play with. These players go by the names of Til Schweiger, Udo Kier and Ralf Moeller.<br /><br />Three names that actually have made them selfs pretty big in the movie biz. So the tale goes like this, Jack Carver played by Til Schweiger (yes Carver is German all hail the bratwurst eating dudes!!) However I find that Tils acting in this movie is pretty badass.. People have complained about how he's not really staying true to the whole Carver agenda but we on

In [22]:

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
"""
min_df=2, discard words appearing in less than 2 documents
max_df=0.9, discard words appering in more than 90% of the documents
sublinear_tf=True, use sublinear weighting
use_idf=True, enable IDF
"""
vec = TfidfVectorizer(
    analyzer=preprocess,
    min_df=2,
    max_df=0.9,
    sublinear_tf=True,
    use_idf=True
)

tfidf_model = vec.fit(data['review'])
train_vec = vec.transform(data['review'])

#save vectorizer
vectorizer_file = r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder/tfidf_vectorizer.sav';
pickle.dump(tfidf_model, open(vectorizer_file, 'wb'))

In [27]:
print(train_vec) #(Document number, word index number)       IDF score

  (0, 44622)	0.051051921991970436
  (0, 44535)	0.09234484225013369
  (0, 43743)	0.03280097979657562
  (0, 43533)	0.06123646425193938
  (0, 43048)	0.15267635106300764
  (0, 42973)	0.05252362389325301
  (0, 42501)	0.041614436536244026
  (0, 41939)	0.12136572627028748
  (0, 41672)	0.08914544599409176
  (0, 41191)	0.044863711267744806
  (0, 41078)	0.07662586879126554
  (0, 40597)	0.059814263228259903
  (0, 40211)	0.11349753657676434
  (0, 39850)	0.035159489203875856
  (0, 39718)	0.07912507567426838
  (0, 39304)	0.0705968807711694
  (0, 38728)	0.08264214651353383
  (0, 38216)	0.15395936198492918
  (0, 38128)	0.06619523576894111
  (0, 37707)	0.0627814143259084
  (0, 37654)	0.08492905949150271
  (0, 36857)	0.08949552918897383
  (0, 36261)	0.07212140727476048
  (0, 35930)	0.058587962965835544
  (0, 35787)	0.08791749541271658
  :	:
  (49999, 21263)	0.20837045635142148
  (49999, 19570)	0.1629510683026242
  (49999, 19169)	0.1094291879482535
  (49999, 19057)	0.18670801891423458
  (49999, 18469)	0.

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
SEED = 4000

X_train, X_test, y_train, y_test = train_test_split(train_vec, 
                                            data.label, test_size=0.2, random_state=SEED)

print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))
print('y_train shape: {}'.format(y_train.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (40000, 45394)
X_test shape: (10000, 45394)
y_train shape: (40000,)
y_test shape: (10000,)


In [31]:
from sklearn.svm import SVC


clf_svm = SVC()

clf_names = ['SVM']

clf_types = [clf_svm]

for (i, clf) in enumerate(clf_types):
    clf.fit(X_train, y_train.values.ravel())
    print('Result of {}\n'.format(clf_names[i]))
    predictions = clf.predict(X_test)
    print(classification_report(predictions, y_test))
    print('\n')
    print('Confusion matrix: \n', confusion_matrix(predictions, y_test))
    print('\n')
    print('Accuracy score: ', accuracy_score(predictions, y_test))
    print('\n\n\n')

Result of SVM

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      4875
           1       0.91      0.89      0.90      5125

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



Confusion matrix: 
 [[4435  440]
 [ 571 4554]]


Accuracy score:  0.8989






In [32]:
filename = r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder\SVM_sentiment_model.sav'
pickle.dump(clf_svm, open(filename, 'wb'))